In [ ]:
import numpy as np
import pandas as pdM
df=pd.read_csv('Sales.csv') #dataset
df.head()

df.shape #rows and coloumns

df.isnull().sum()

Gender = pd.get_dummies(df, columns=['Gender'])

Gender.head()

X=df.iloc[:,[2,3]] #variable 
Y=df.iloc[:,[4]]   #target 

X.head()

X.describe()

Y.Purchased.value_counts(normalize=True)

#split 
from sklearn.model_selection import train_test_split
X_Train,X_Test,Y_Train,Y_Test=train_test_split(X,Y,test_size=0.25,random_state=0)

print("Training data: ",X_Train.shape)
print("Test data: ",X_Test.shape)

#feature

from sklearn.preprocessing import StandardScaler # to be units and precision
sc_X=StandardScaler()
X_Train=sc_X.fit_transform(X_Train) #transformation,mean and standard deviation from the training data during the fit step.
X_Test=sc_X.transform(X_Test) 

#scatter plot train data , purple 0 yellow 1 
import matplotlib.pyplot as plt

plt.scatter(X_Train[:, 0], X_Train[:, 1],c=Y_Train.values.ravel()) 
plt.xlabel('Age')
plt.ylabel('Estimated salary') 
plt.title('Train Data') 
plt.show()

#scatter plot test data
import matplotlib.pyplot as plt

plt.scatter(X_Test[:, 0], X_Test[:, 1],c=Y_Test.values.ravel()) 
plt.xlabel('Age')
plt.ylabel('Estimated salary') 
plt.title('Test Data') 
plt.show()

##model
from sklearn.svm import SVC #algo
#a kernel is a function that computes a similarity measure between pairs of data points in a given feature space.
classifier=SVC(kernel='linear',random_state=0)  #hyperplane separates the different classes of data points
classifier.fit(X_Train,Y_Train.values.ravel()) #train

#predicting results
Y_Pred=classifier.predict(X_Test) 

Y_Pred

n=Y_Pred.shape[0]
n

from sklearn import metrics
print('Accuracy(linear):',metrics.accuracy_score(Y_Test,Y_Pred))

#hyperplane
# plot datapoints  
plt.scatter(X_Test[:, 0],X_Test[:, 1],c=Y_Test.values.ravel()) 
#plt.scatter (x Train:, 0), x Train:, 1), Train)


# Create the hyperplane
w = classifier.coef_[0]
a = -w[0] / w[1]
xx = np.linspace(-2.5, 2.5)
yy = a * xx - (classifier.intercept_[0]) / w[1]

# Plot the hyperplane
plt.plot(xx, yy)

# Show the plot
plt.show()


#confusion metrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_Test,Y_Pred)

print ('Confusion Matrix(GaussianNB)')
cm

TN = cm[0,0]
print("True Negative:",TN)

FP=cm[0,1]
print("False Possitive:",FP)

FN = cm[1,0]
print("False Negative:",FN)

TP = cm[1,1]
print("True Possitive:",TP)

#Again Accuracy
(TN+TP)/n
#TPR(True possitive rate) or Sensitivity
recall = TP/(TP+FP)
recall`

#TNR/Specificity
specificity = TN/(TN+FP)
specificity

#FPR
FPR = 1-specificity
FPR
precision = TP/(TP+FP)
precision

#F1_score is the hermonic mean between precision and reall
F1_score = (2*precision*recall)/(precision+recall)
F1_score




#or
from sklearn.metrics import f1_score
f1_score(Y_Test,Y_Pred)


## ROC(Receiver operating characteristic) Curve 
#[plot between false positive rate and true positive rate]
from sklearn.metrics import roc_curve
probs = classifier.decision_function(X_Test)
fpr, tpr, threshold = roc_curve(Y_Test, probs)

# Reshape each array individually
threshold_reshaped = threshold.reshape(-1, 1)
fpr_reshaped = fpr.reshape(-1, 1)
tpr_reshaped = tpr.reshape(-1, 1)

# Horizontally stack the reshaped arrays
stacked_data = np.hstack((threshold_reshaped, fpr_reshaped, tpr_reshaped))

# Create a DataFrame with appropriate column names
df = pd.DataFrame(stacked_data, columns=['threshold', 'fpr', 'tpr'])

df.head()


# from sklearn.metrics import roc_curve
# probs = classifier.predict_proba(X_Test)
# probs
# prob_positive = probs[:, 1]
# fpr, tpr, threshold = roc_curve(Y_Test, prob_positive)

# # Reshape each array individually
# threshold_reshaped = threshold.reshape(-1, 1)
# fpr_reshaped = fpr.reshape(-1, 1)
# tpr_reshaped = tpr.reshape(-1, 1)

# # Horizontally stack the reshaped arrays
# stacked_data = np.hstack((threshold_reshaped, fpr_reshaped, tpr_reshaped))

# # Create a DataFrame with appropriate column names
# df = pd.DataFrame(stacked_data, columns=['threshold', 'fpr', 'tpr'])

# df.head()



from sklearn.metrics import auc
roc_auc = auc(fpr,tpr)
roc_auc
import matplotlib.pyplot as plt
plt.title("receicer operating charecteristic")
plt.plot(fpr,tpr,'red',label = 'AUC-%0.2f'% roc_auc)
plt.legend(loc = 'lower right')

plt.plot([0,1],[0,1], color = 'darkblue',linestyle='--')
plt.ylabel('True positive rate')
plt.xlabel('False positive rate')
plt.show()

#F1 flot

thresholds = np.arange(0, 1.01, 0.01)  # Generate thresholds from 0 to 1
f1_scores = [f1_score(Y_Test, Y_Pred > threshold) for threshold in thresholds]

plt.figure(figsize=(8, 6))
plt.plot(thresholds, f1_scores, label='F1 Score')
plt.xlabel('Threshold')
plt.ylabel('F1 Score')
plt.title('F1 Score Curve')
plt.legend(loc='lower left')
plt.show()

#Second

from sklearn.svm import SVC
classifier_Rbf=SVC(kernel='rbf') #radial basis function
classifier_Rbf.fit(X_Train,Y_Train.values.ravel())

#predicting results
Y_Pred=classifier_Rbf.predict(X_Test)

print('Accuracy(rbf):',metrics.accuracy_score(Y_Test,Y_Pred))

# Create a meshgrid for plotting the decision boundary
h = 0.02
x_min, x_max = X_Test[:, 0].min() - 1, X_Test[:, 0].max() + 1
y_min, y_max = X_Test[:, 1].min() - 1, X_Test[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

# Predict class labels for the meshgrid points
Z = classifier.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

# Plot data points and decision boundary
plt.contourf(xx, yy, Z, cmap=plt.cm.Spectral, alpha=0.8)
plt.scatter(X_Test[:, 0], X_Test[:, 1], c=Y_Test.values.ravel(), cmap=plt.cm.Spectral)
plt.xlim(xx.min(), xx.max())
plt.ylim(yy.min(), yy.max())
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title('SVM with RBF Kernel')
plt.show()


#confusion metrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_Test,Y_Pred)

print ('Confusion Matrix(GaussianNB)')
cm

TN = cm[0,0]
print("True Negative:",TN)

FP=cm[0,1]
print("False Possitive:",FP)

FN = cm[1,0]
print("False Negative:",FN)

TP = cm[1,1]
print("True Possitive:",TP)

#Again Accuracy
(TN+TP)/n
#TPR(True possitive rate) or Sensitivity
recall = TP/(TP+FP)
recall

#TNR/Specificity
specificity = TN/(TN+FP)
specificity

#FPR
FPR = 1-specificity
FPR
precision = TP/(TP+FP)
precision

#F1_score is the hermonic mean between precision and reall
F1_score = (2*precision*recall)/(precision+recall)
F1_score


#or
from sklearn.metrics import f1_score
f1_score(Y_Test,Y_Pred)

## ROC(Receiver operating characteristic) Curve 
#[plot between false positive rate and true positive rate]
from sklearn.metrics import roc_curve
probs = classifier_Rbf.decision_function(X_Test)
fpr, tpr, threshold = roc_curve(Y_Test, probs)

# Reshape each array individually
threshold_reshaped = threshold.reshape(-1, 1)
fpr_reshaped = fpr.reshape(-1, 1)
tpr_reshaped = tpr.reshape(-1, 1)

# Horizontally stack the reshaped arrays
stacked_data = np.hstack((threshold_reshaped, fpr_reshaped, tpr_reshaped))

# Create a DataFrame with appropriate column names
df = pd.DataFrame(stacked_data, columns=['threshold', 'fpr', 'tpr'])

df.head()

from sklearn.metrics import auc
roc_auc = auc(fpr,tpr)
roc_auc
import matplotlib.pyplot as plt
plt.title("receicer operating charecteristic")
plt.plot(fpr,tpr,'red',label = 'AUC-%0.2f'% roc_auc)
plt.legend(loc = 'lower right')

plt.plot([0,1],[0,1], color = 'darkblue',linestyle='--')
plt.ylabel('True positive rate')
plt.xlabel('False positive rate')
plt.show()

#F1 flot

thresholds = np.arange(0, 1.01, 0.01)  # Generate thresholds from 0 to 1
f1_scores = [f1_score(Y_Test, Y_Pred > threshold) for threshold in thresholds]

plt.figure(figsize=(8, 6))
plt.plot(thresholds, f1_scores, label='F1 Score')
plt.xlabel('Threshold')
plt.ylabel('F1 Score')
plt.title('F1 Score Curve')
plt.legend(loc='lower left')
plt.show()


#third
sv=SVC(kernel='poly',degree=4)
sv.fit(X_Train,Y_Train.values.ravel())

#predicting results
Y_Pred=sv.predict(X_Test)

print('Accuracy(poly):',metrics.accuracy_score(Y_Test,Y_Pred))

# Create a meshgrid for plotting the decision boundary
h = 0.02
x_min, x_max = X_Test[:, 0].min() - 1, X_Test[:, 0].max() + 1
y_min, y_max = X_Test[:, 1].min() - 1, X_Test[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

# Predict class labels for the meshgrid points
Z = classifier.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

# Plot data points and decision boundary
plt.contourf(xx, yy, Z, cmap=plt.cm.Spectral, alpha=0.8)
plt.scatter(X_Test[:, 0], X_Test[:, 1], c=Y_Test.values.ravel(), cmap=plt.cm.Spectral)
plt.xlim(xx.min(), xx.max())
plt.ylim(yy.min(), yy.max())
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title('SVM with Polynomial Kernel (Degree 4)')
plt.show()


#confusion metrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_Test,Y_Pred)

print ('Confusion Matrix(GaussianNB)')
cm

TN = cm[0,0]
print("True Negative:",TN)

FP=cm[0,1]
print("False Possitive:",FP)

FN = cm[1,0]
print("False Negative:",FN)

TP = cm[1,1]
print("True Possitive:",TP)

#Again Accuracy
(TN+TP)/n
#TPR(True possitive rate) or Sensitivity
recall = TP/(TP+FP)
recall

#TNR/Specificity
specificity = TN/(TN+FP)
specificity

#FPR
FPR = 1-specificity
FPR
precision = TP/(TP+FP)
precision

#F1_score is the hermonic mean between precision and reall
F1_score = (2*precision*recall)/(precision+recall)
F1_score

#or
from sklearn.metrics import f1_score
f1_score(Y_Test,Y_Pred)

## ROC(Receiver operating characteristic) Curve 
#[plot between false positive rate and true positive rate]
from sklearn.metrics import roc_curve
probs = sv.decision_function(X_Test)
fpr, tpr, threshold = roc_curve(Y_Test, probs)

# Reshape each array individually
threshold_reshaped = threshold.reshape(-1, 1)
fpr_reshaped = fpr.reshape(-1, 1)
tpr_reshaped = tpr.reshape(-1, 1)

# Horizontally stack the reshaped arrays
stacked_data = np.hstack((threshold_reshaped, fpr_reshaped, tpr_reshaped))

# Create a DataFrame with appropriate column names
df = pd.DataFrame(stacked_data, columns=['threshold', 'fpr', 'tpr'])

df.head()

from sklearn.metrics import auc
roc_auc = auc(fpr,tpr)
roc_auc
import matplotlib.pyplot as plt
plt.title("receicer operating charecteristic")
plt.plot(fpr,tpr,'red',label = 'AUC-%0.2f'% roc_auc)
plt.legend(loc = 'lower right')

plt.plot([0,1],[0,1], color = 'darkblue',linestyle='--')
plt.ylabel('True positive rate')
plt.xlabel('False positive rate')
plt.show()

#F1 flot

thresholds = np.arange(0, 1.01, 0.01)  # Generate thresholds from 0 to 1
f1_scores = [f1_score(Y_Test, Y_Pred > threshold) for threshold in thresholds]

plt.figure(figsize=(8, 6))
plt.plot(thresholds, f1_scores, label='F1 Score')
plt.xlabel('Threshold')
plt.ylabel('F1 Score')
plt.title('F1 Score Curve')
plt.legend(loc='lower left')
plt.show()

#fourth

from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()
nb.fit(X_Train, Y_Train.values.ravel())

Y_pred = nb.predict(X_Test)

print('Accuracy(GaussianNB):', metrics.accuracy_score(Y_Test, Y_Pred))



# Create a meshgrid for plotting the decision boundary
h = 0.02
x_min, x_max = X_Test[:, 0].min() - 1, X_Test[:, 0].max() + 1
y_min, y_max = X_Test[:, 1].min() - 1, X_Test[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

# Predict class labels for the meshgrid points
Z = classifier.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

# Plot data points and decision boundary
plt.contourf(xx, yy, Z, cmap=plt.cm.Spectral, alpha=0.8)
plt.scatter(X_Test[:, 0], X_Test[:, 1], c=Y_Test.values.ravel(), cmap=plt.cm.Spectral)
plt.xlim(xx.min(), xx.max())
plt.ylim(yy.min(), yy.max())
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title('Gaussian')
plt.show()


#seond part 
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_Test,Y_Pred)

print ('Confusion Matrix(GaussianNB)')
cm

TN = cm[0,0]
print("True Negative:",TN)

FP=cm[0,1]
print("False Possitive:",FP)

FN = cm[1,0]
print("False Negative:",FN)

TP = cm[1,1]
print("True Possitive:",TP)

#Again Accuracy
(TN+TP)/n
#TPR(True possitive rate) or Sensitivity
recall = TP/(TP+FP)
recall

#TNR/Specificity
specificity = TN/(TN+FP)
specificity

#FPR
FPR = 1-specificity
FPR
precision = TP/(TP+FP)
precision

#F1_score is the hermonic mean between precision and reall
F1_score = (2*precision*recall)/(precision+recall)
F1_score

#or
from sklearn.metrics import f1_score
f1_score(Y_Test,Y_Pred)

## ROC(Receiver operating characteristic) Curve 
#[plot between false positive rate and true positive rate]
from sklearn.metrics import roc_curve
probs = nb.predict_proba(X_Test)
probs
prob_positive = probs[:, 1]
fpr, tpr, threshold = roc_curve(Y_Test, prob_positive)



# Reshape each array individually
threshold_reshaped = threshold.reshape(-1, 1)
fpr_reshaped = fpr.reshape(-1, 1)
tpr_reshaped = tpr.reshape(-1, 1)


# Horizontally stack the reshaped arrays
stacked_data = np.hstack((threshold_reshaped, fpr_reshaped, tpr_reshaped))

# Create a DataFrame with appropriate column names
df = pd.DataFrame(stacked_data, columns=['threshold', 'fpr', 'tpr'])

df.head()


from sklearn.metrics import auc
roc_auc = auc(fpr,tpr)
roc_auc
import matplotlib.pyplot as plt
plt.title("receicer operating charecteristic")
plt.plot(fpr,tpr,'red',label = 'AUC-%0.2f'% roc_auc)
plt.legend(loc = 'lower right')

plt.plot([0,1],[0,1], color = 'darkblue',linestyle='--')
plt.ylabel('True positive rate')
plt.xlabel('False positive rate')
plt.show()

#F1 flot

thresholds = np.arange(0, 1.01, 0.01)  # Generate thresholds from 0 to 1
f1_scores = [f1_score(Y_Test, Y_Pred > threshold) for threshold in thresholds]

plt.figure(figsize=(8, 6))
plt.plot(thresholds, f1_scores, label='F1 Score')
plt.xlabel('Threshold')
plt.ylabel('F1 Score')
plt.title('F1 Score Curve')
plt.legend(loc='lower left')
plt.show()